# Install and import lib

In [ ]:
%pip install tweety-ns
%pip install https://github.com/mahrtayyab/tweety/archive/main.zip --upgrade 

In [1]:
import asyncio
import csv
from twitter_search import read_keywords_from_file, run_search_user, filter_users, login,crawl_tweets_from_KOLs, run_crawl_user_profile

# Login


In [2]:
app = await login()

In [4]:
namelists = read_keywords_from_file("outputKOL.txt")
await run_crawl_user_profile(namelists, app)

Appended users 1025Hourai to KOL_profile.csv


UserNotFound: The User Account wasn't Found or is Protected

# Search user by keyword and save to csv

In [8]:



# Run the search and save function
keywords = read_keywords_from_file("keywords.txt")
await run_search_user(keywords, app, pages_num = 30, wait_times = 10, cursor_ = None) #cusor is the page that looking up-to

Appended 11 users to twitter_users_Lending Protocols.csv
Appended 600 users to twitter_users_Stablecoin.csv


# filter user to KOLs

In [3]:
keywords = read_keywords_from_file("keywords.txt")
filter_users(keywords)

Keyword 'EVM': 1 users remaining after filtering.
Keyword 'Ethereum Virtual Machine': 0 users remaining after filtering.
Keyword 'Cross-chain': 4 users remaining after filtering.
Keyword 'Tokenomics': 3 users remaining after filtering.
Keyword 'zk-rollup': 0 users remaining after filtering.
Keyword 'Proof of Stake': 0 users remaining after filtering.
Keyword 'Proof of Work': 2 users remaining after filtering.
Keyword 'Layer-1 Protocols': 0 users remaining after filtering.
Keyword 'Layer-2 Protocols': 0 users remaining after filtering.
Keyword 'Liquidity Mining': 0 users remaining after filtering.
Keyword 'Yield Farming': 3 users remaining after filtering.
Keyword 'Automated Market Maker': 0 users remaining after filtering.
Keyword 'Decentralized Exchanges': 2 users remaining after filtering.
Keyword 'DEX': 2 users remaining after filtering.
Keyword 'Staking Rewards': 0 users remaining after filtering.
Keyword 'Lending Protocols': 0 users remaining after filtering.
Keyword 'Stablecoin':

'filtered_twitter_users.csv'

# Crawl tweet


In [ ]:
# # Basic Tweet Information
# 'id': int                # The unique ID of the tweet (assigned by Twitter).
# 'created_on': datetime   # The date and time when the tweet was created.
# 'date': datetime         # The date and time when the tweet was created (same as created_on).

# # Main Content of the Tweet
# 'text': str              # The text content of the tweet (original text).
# 'rich_text': RichText    # The rich text content (if any), which may include HTML formatting or other styles.
# 'author': User           # The user object containing information about the tweet's author (e.g., username, ID, etc.).

# # Tweet Interaction (Retweets, Quotes, Replies)
# 'is_retweet': bool              # If `True`, the tweet is a retweet of another tweet.
# 'retweeted_tweet_id': int       # The ID of the original tweet that was retweeted (if applicable).
# 'is_quoted': bool               # If `True`, the tweet is a quoted tweet.
# 'quoted_tweet_id': int          # The ID of the tweet that was quoted (if applicable).
# 'is_reply': bool                # If `True`, the tweet is a reply to another tweet.
# 'replied_to': Tweet | str       # The tweet being replied to (if applicable), or the user ID if it is a reply to a user.

# # Sensitive Content Information
# 'is_sensitive': bool            # If `True`, the tweet contains sensitive content (e.g., graphic images, NSFW).

# # Engagement Metrics
# 'reply_counts': int            # The number of times someone has replied to this tweet.
# 'quote_counts': int            # The number of times this tweet has been quoted.
# 'bookmark_count': int          # The number of times this tweet has been bookmarked.
# 'views': int                   # The number of times this tweet has been viewed.
# 'likes': int                   # The number of likes (favorites) this tweet has received.
# 'language': str                # The language of the tweet as identified by Twitter (e.g., 'en' for English).
# 'place': Place                 # The location or place associated with the tweet, if available.

# # Retweet Information
# 'retweet_counts': int          # The number of times this tweet has been retweeted.

# # Source and Moderation Information
# 'source': str                  # The source application used to post the tweet (e.g., "Twitter Web App").
# 'has_moderated_replies': bool  # If `True`, replies to the tweet are moderated by the author.

# # User Interaction Information
# 'is_liked': bool               # If `True`, the authenticated user has liked this tweet.
# 'is_retweeted': bool           # If `True`, the authenticated user has retweeted this tweet.
# 'can_reply': bool              # If `True`, the authenticated user is allowed to reply to this tweet.

# # Broadcast and Edit Information
# 'broadcast': Broadcast | None  # Information about the broadcast (if the tweet is part of a broadcast).
# 'edit_control': EditControl | None  # Information about the tweet's edit control (if it has been edited).
# 'has_newer_version': bool      # If `True`, this tweet has been edited, and there is a newer version of it.

# # Additional Tweet Metadata
# 'audio_space_id': str          # The ID of the Audio Space associated with the tweet (if applicable).
# 'pool': Pool | None            # The pool associated with the tweet (if applicable).
# 'community': Community | None  # The community this tweet is a part of (if applicable).

# # Media and Mentions
# 'media': list[Media]          # A list of media objects (e.g., images, videos) attached to the tweet.
# 'user_mentions': list[ShortUser]  # A list of users mentioned in the tweet.
# 'urls': list[URL]             # A list of URLs mentioned in the tweet.
# 'hashtags': list[Hashtag]      # A list of hashtags mentioned in the tweet.
# 'symbols': list[Symbol]       # A list of symbols (e.g., stock symbols) mentioned in the tweet.

# # Community and Thread Information
# 'community_note': str | None   # A community note posted in response to the tweet (if applicable).
# 'url': str                     # The URL of the tweet.
# 'threads': list[Tweet]         # A list of tweets in the thread of this tweet.
# 'comments': list[ConversationThread]  # A list of replies or comments sent in response to this tweet.


In [4]:

input_file_path = "filtered_twitter_users.csv"
output_file_path ="final_tweet.csv"
await crawl_tweets_from_KOLs(input_file_path, output_file_path, app, pages=10, replies=False, wait_time=2, cursor=None)

Fetching tweets for user: JSDRAM
Saved 18 tweets to final_tweet.csv.


In [ ]:
user profile